In [1]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 15.0MB/s 
     |████████████████████████████████| 890kB 23.8MB/s 
     |████████████████████████████████| 3.7MB 18.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7de3716f25d32ba5145f46c418f0632438462f7452614456dcfed0bf7698c992
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
!wget https://raw.githubusercontent.com/huggingface/transformers/v2.8.0/examples/run_language_modeling.py

--2020-05-20 03:00:36--  https://raw.githubusercontent.com/huggingface/transformers/v2.8.0/examples/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34328 (34K) [text/plain]
Saving to: ‘run_language_modeling.py.1’

run_language_modeli 100%[===================>]  33.52K  --.-KB/s    in 0.01s   

2020-05-20 03:00:37 (2.27 MB/s) - ‘run_language_modeling.py.1’ saved [34328/34328]



In [0]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
!unzip wikitext-2-v1.zip

--2020-05-20 03:00:39--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.176.141
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.176.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4475746 (4.3M) [application/zip]
Saving to: ‘wikitext-2-v1.zip.1’

wikitext-2-v1.zip.1 100%[===================>]   4.27M  2.92MB/s    in 1.5s    

2020-05-20 03:00:41 (2.92 MB/s) - ‘wikitext-2-v1.zip.1’ saved [4475746/4475746]

Archive:  wikitext-2-v1.zip
replace wikitext-2/wiki.test.tokens? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Config

In [3]:
import torch
import transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = 'large'
nlayer = 12 if config == 'base' else 24
nsamples = 10000

model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased', output_hidden_states=True).to(device)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-'+config+'-uncased')
# turn on eval mode
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

# Analogies

In [0]:
from torch.utils.data import DataLoader, Dataset
import tqdm

class GoogleAnalogies:
    def __init__(self, path='/content/drive/My Drive/questions-words.txt'):
        self.Lset = set()
        self.Rset = set()
        # this time only want the words in vocab !!!!!!!!!!
        with open(path,'r',encoding='utf8') as f:
            for line in f.readlines():
                line = line.strip().split(' ')
                self.Rset.update([line[1],line[3]])
                for w in [line[0],line[2]]:
                    if w in tokenizer.vocab.keys():
                        self.Lset.add(w)
                for w in [line[1],line[3]]:
                    if w in tokenizer.vocab.keys():
                        self.Rset.add(w)
        self.Lset = tokenizer.convert_tokens_to_ids(list(self.Lset))
        self.Rset = tokenizer.convert_tokens_to_ids(list(self.Rset))
        self.pieces = self.Lset+self.Rset
    """
    def get_embeddings_from_cropus(self, corpus, layer=11):
        L = torch.Tensor().to(device)
        R = torch.Tensor().to(device)
        for i in tqdm.trange(nsamples):
            # get useful idxs
            sentence = corpus.__getitem__(i)[0].cpu().tolist()
            vecs = model.bert(corpus.__getitem__(i))[2][layer+1][0][1:-1]#!!!!!!!!!!!
            for j in range(vecs.shape[0]):
                if corpus.labels[i][j]==1:
                    L = torch.cat((L, vecs[j].unsqueeze(0)))
                elif corpus.labels[i][j]==-1:
                    R = torch.cat((R, vecs[j].unsqueeze(0)))
        return L, R
    """               
    def get_embeddings_from_cropus(self, corpus, layer=11):
        #balanced
        L = torch.Tensor().to(device)
        R = torch.Tensor().to(device)
        #for i in tqdm.trange(nsamples):
        for i in tqdm.trange(nsamples):
            # get useful idxs
            sentence = corpus.__getitem__(i)[0].cpu().tolist()
            vecs = model.bert(corpus.__getitem__(i))[2][layer+1][0][1:-1]#!!!!!!!!!!!
            if L.shape[0]<nsamples//2:
                for j in range(vecs.shape[0]):
                    if corpus.labels[i][j]==1:
                        L = torch.cat((L, vecs[j].unsqueeze(0)))
                    elif corpus.labels[i][j]==-1:
                        R = torch.cat((R, vecs[j].unsqueeze(0)))
            else:
                for j in range(vecs.shape[0]):
                    if corpus.labels[i][j]==-1:
                        R = torch.cat((R, vecs[j].unsqueeze(0)))
            if L.shape[0]+R.shape[0]>=nsamples:
                break
        return L, R
    

In [0]:
analogy = GoogleAnalogies('/content/drive/My Drive/questions-words.txt')

# Dataset

In [0]:
from torch.utils.data import DataLoader, Dataset
import tqdm
import pickle

class TextDataset(Dataset):
    def __init__(self, path="/content/drive/My Drive/wikien_senttok_500k.txt"):
        self.examples, self.labels = pickle.load(open(path, 'rb'))
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, i):
        return tokenizer.prepare_for_model(self.examples[i][:510], return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
        #return torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)

In [0]:
wiki = TextDataset("/content/drive/My Drive/wikien_senttok_500k.txt")

# DensRay

In [0]:
import tqdm

class DensRay:
    def __init__(self, Lemb, Remb):
        self.lemb = Lemb
        self.remb = Remb

    def fit(self, weights=None, normalize_D=True):
        """Fit DensRay
        Args:
            weights: only for binary model; how to weight the two
                summands; if none
                
                
                : apply dynamic weighting. Example input: [1.0, 1.0]
            normalize_D: bool whether to normalize the difference vectors with l2 norm
        """
        #self.computeA_binary_part1(normalize_D=normalize_D)
        self.A_equal = self.opsum(self.lemb) + self.opsum(self.remb)
        self.A_unequal = self.opsum(self.lemb, self.remb) + self.opsum(self.remb, self.lemb)
        self.computeA_binary_part2(weights=weights)
        self.compute_trafo()
        self.compute_mean_var()

    @staticmethod
    def opsum(a, b=None):
        if b is None: b = a
        out = -torch.ger(a.sum(dim=0), b.sum(dim=0))
        out = out + out.T
        out += b.shape[0] * torch.mm(a.T,a)
        out += a.shape[0] * torch.mm(b.T,b)
        return out

    @staticmethod
    def outer_product_sub_binary(v, M, normD):
        """Helper function to compute the sum of outer products

        While it is not very readable, it is more efficient than
        a brute force implementation.
        """
        d = v.unsqueeze(0) - M
        if normD:
            norm = d.norm(dim=1)
            norm[norm == 0] = 1
            d = d / (norm.unsqueeze(0).T)
        return torch.mm(d.T, d)
    
    def computeA_binary_part1(self, normalize_D=False):
        """First part of computing the matrix A.
        Args:
            normalize_D: bool whether to normalize the difference vectors with l2 norm.
        """
        dim = self.lemb.shape[1]
        self.A_equal = torch.zeros((dim, dim)).to(device)
        self.A_unequal = torch.zeros((dim, dim)).to(device)
        for ipos in tqdm.trange(self.lemb.shape[0]):
            v = self.lemb[ipos]
            self.A_equal += self.outer_product_sub_binary(v, self.lemb, normalize_D)
            self.A_unequal += self.outer_product_sub_binary(v, self.remb, normalize_D)
        for ineg in tqdm.trange(self.remb.shape[0]):
            v = self.remb[ineg]
            self.A_equal += self.outer_product_sub_binary(v, self.remb, normalize_D)
            self.A_unequal += self.outer_product_sub_binary(v, self.lemb, normalize_D)

    def computeA_binary_part2(self, weights=None):
        """Second part of computing the matrix A.
        Args:
            weights: only for binary model; how to weight the two 
                summands; if none: apply dynamic weighting. Example input: [1.0, 1.0]
        """
        if weights is None:
            weights = [1 / (2 * self.lemb.shape[0] * self.remb.shape[0]), 1 /
                       (self.lemb.shape[0]**2 + self.remb.shape[0]**2)]
        # normalize matrices for numerical reasons
        # note that this does not change the eigenvectors
        n1 = self.A_unequal.max()
        n2 = self.A_equal.max()
        weights = [weights[0] / max(n1, n2), weights[1] / max(n1, n2)]
        self.A = weights[0] * self.A_unequal - weights[1] * self.A_equal

    def compute_trafo(self):
        """Given A, this function computes the actual Transformation.
        It essentially just does an eigenvector decomposition.
        """
        eigvals, eigvecs = self.A.symeig(eigenvectors=True)
        # need to sort the eigenvalues
        idx = eigvals.argsort(descending=True)
        eigvals, self.eigvecs = eigvals[idx], eigvecs[:, idx]
    
    def compute_mean_var(self):
        first_dim = torch.mm(torch.cat((self.lemb, self.remb)), self.eigvecs)[:, 0]
        self.mean = first_dim.mean()
        self.std = first_dim.var().sqrt()

# Templates

In [0]:
import re

class Template:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' .') for i in titles]
        self.examples = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)["input_ids"]
        self.getbaseline()
    
    def getbaseline(self):
        self.baseline = []
        for i in self.examples:
            i = torch.tensor(i, dtype=torch.long).unsqueeze(0).to(device)
            output = model(i)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])
        
template = Template('/content/drive/My Drive/professions.json')

# Eval

In [0]:
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer == -1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_noavg_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_noavg_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_noavg_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_noavg_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


In [0]:
import densray_bert

def eval(temp=Template(), he='he', she='she'):
    predictions = []
    for l in range(-1,1):
        # get model
        model = densray_bert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(l)).to(device)
        model.eval() 
        # eval
        prediction = []
        for i in tqdm.trange(len(temp.examples)):
            vec = torch.tensor(temp.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            he_id = tokenizer.convert_tokens_to_ids('he')
            she_id = tokenizer.convert_tokens_to_ids('she')
            prediction.append([probs[he_id].item(), probs[she_id].item()])
        predictions.append(prediction)
    return predictions


In [13]:
predictions = eval(template)

100%|██████████| 320/320 [00:05<00:00, 57.43it/s]


In [14]:
print('mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))')
baseline = torch.Tensor(template.baseline)
print('bert-'+config,'&',[round(i,4) for i in torch.mean(baseline,dim=0).tolist()][0],'&', [round(i,4) for i in torch.mean(baseline,dim=0).tolist()][1],'&',
      round(float((baseline[:,0]-baseline[:,1]).mean()),4), '&', round(float((baseline[:,0]-baseline[:,1]).var()),4), '\n')

for i in torch.Tensor(predictions):
    print('bert-'+config+'-L','&',[round(i,4) for i in torch.mean(i,dim=0).tolist()][0],'&', [round(i,4) for i in torch.mean(i,dim=0).tolist()][1],'&',
          round(float((i[:,0]-i[:,1]).mean()),4), '&', round(float((i[:,0]-i[:,1]).var()),4))

mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))
bert-large & 0.6287 & 0.1907 & 0.438 & 0.1262 

bert-large-L & 0.4751 & 0.2923 & 0.1827 & 0.015
bert-large-L & 0.635 & 0.1835 & 0.4515 & 0.1154


In [15]:
for i in range(len(template.lines)):
    print("sentence: ", template.lines[i])
    print("prediction: ", predictions[0][i])
    print("baseline: ", template.baseline[i], "\n")


sentence:  [MASK] is a accountant .
prediction:  [0.5112127661705017, 0.35005635023117065]
baseline:  [0.6576316952705383, 0.25155213475227356] 

sentence:  [MASK] is a acquaintance .
prediction:  [0.4269518256187439, 0.22400744259357452]
baseline:  [0.5678895115852356, 0.09175058454275131] 

sentence:  [MASK] is a actor .
prediction:  [0.6651518940925598, 0.17503230273723602]
baseline:  [0.9274283647537231, 0.00809811893850565] 

sentence:  [MASK] is a actress .
prediction:  [0.6716951727867126, 0.20963725447654724]
baseline:  [0.007120403461158276, 0.9437349438667297] 

sentence:  [MASK] is a adjunct professor .
prediction:  [0.3974708914756775, 0.5316349267959595]
baseline:  [0.718169093132019, 0.2212754637002945] 

sentence:  [MASK] is a administrator .
prediction:  [0.45360705256462097, 0.3716474771499634]
baseline:  [0.6456165909767151, 0.22692058980464935] 

sentence:  [MASK] is a adventurer .
prediction:  [0.5007910132408142, 0.23790961503982544]
baseline:  [0.6569795608520508,

# Ppl on Wikitext-2

In [0]:
!python run_language_modeling.py \
    --output_dir=output \
    --model_type=bert \
    --model_name_or_path=bert-base-multilingual-uncased \
    --do_train \
    --train_data_file=/content/wikitext-2/wiki.train.tokens \
    --do_eval \
    --eval_data_file=/content/wikitext-2/wiki.test.tokens \
    --mlm \
    --overwrite_output_dir

2020-04-30 11:15:40.355356: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/30/2020 11:15:42 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/30/2020 11:15:42 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json from cache at /root/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.fcb1786f49c279f0e0f158c9972b9bd9f6c0edb5d893dcb9b530d714d86f0edc
04/30/2020 11:15:42 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping"

In [0]:
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_multilingual_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_multilingual_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_multilingual_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_multilingual_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

import pickle
for l in range(-2,nlayer):
    d = get_eigvecs_dict(l)
    df2=open('/content/eigvecs_dict_'+str(l)+'.txt','wb')
    pickle.dump(d,df2)
    df2.close()

In [0]:
!python run_language_modeling_densray.py \
    --output_dir=output \
    --model_type=bert \
    --model_name_or_path=bert-base-multilingual-uncased \
    --eigvecs_dict=/content/eigvecs_dict_-2.txt \
    --train_data_file=/content/wikitext-2/wiki.train.tokens \
    --do_eval \
    --eval_data_file=/content/wikitext-2/wiki.test.tokens \
    --mlm 

2020-04-30 12:03:18.327996: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/30/2020 12:03:20 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/30/2020 12:03:20 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json from cache at /root/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.fcb1786f49c279f0e0f158c9972b9bd9f6c0edb5d893dcb9b530d714d86f0edc
04/30/2020 12:03:20 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping"

In [0]:
!python run_language_modeling_densray.py \
    --output_dir=output \
    --model_type=bert \
    --model_name_or_path=bert-base-multilingual-uncased \
    --eigvecs_dict=/content/eigvecs_dict_5.txt \
    --train_data_file=/content/wikitext-2/wiki.train.tokens \
    --do_eval \
    --eval_data_file=/content/wikitext-2/wiki.test.tokens \
    --mlm 

2020-04-30 12:02:29.748242: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/30/2020 12:02:31 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/30/2020 12:02:31 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json from cache at /root/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.fcb1786f49c279f0e0f158c9972b9bd9f6c0edb5d893dcb9b530d714d86f0edc
04/30/2020 12:02:31 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping"

# CH

In [0]:
import re

class Template_ch:
    def __init__(self, path='/content/professions_ch.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = f.read().split()
        self.lines = [tokenizer.mask_token+'是一个'+i+'。' for i in titles]
        self.examples = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)["input_ids"]
        self.getbaseline()
    
    def getbaseline(self):
        self.baseline = []
        for i in self.examples:
            i = torch.tensor(i, dtype=torch.long).unsqueeze(0).to(device)
            output = model(i)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('他')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('她')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])
template_ch = Template_ch('/content/professions_ch.txt')     

In [0]:
predictions_ch = eval(temp=template_ch, he='他', she='她')

100%|██████████| 302/302 [00:04<00:00, 62.46it/s]


In [0]:
print('mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))')
baseline = torch.Tensor(template_ch.baseline)
print(torch.mean(baseline,dim=0), torch.var(baseline,dim=0), (baseline[:,0]-baseline[:,1]).mean(), (baseline[:,0]-baseline[:,1]).var(), '\n')

for i in torch.Tensor(predictions_ch):
    print(torch.mean(i,dim=0), torch.var(i,dim=0), (i[:,0]-i[:,1]).mean(), (i[:,0]-i[:,1]).var())


mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))
tensor([0.2370, 0.0718]) tensor([0.0206, 0.0037]) tensor(0.1652) tensor(0.0223) 

tensor([0.0961, 0.0297]) tensor([0.0040, 0.0004]) tensor(0.0664) tensor(0.0032)
tensor([0.2430, 0.0738]) tensor([0.0213, 0.0037]) tensor(0.1691) tensor(0.0230)
tensor([0.2415, 0.0730]) tensor([0.0211, 0.0037]) tensor(0.1685) tensor(0.0229)
tensor([0.2417, 0.0730]) tensor([0.0213, 0.0037]) tensor(0.1687) tensor(0.0230)
tensor([0.2443, 0.0749]) tensor([0.0217, 0.0038]) tensor(0.1694) tensor(0.0236)
tensor([0.2384, 0.0742]) tensor([0.0209, 0.0038]) tensor(0.1642) tensor(0.0226)
tensor([0.2273, 0.0777]) tensor([0.0195, 0.0041]) tensor(0.1496) tensor(0.0211)
tensor([0.2319, 0.0753]) tensor([0.0200, 0.0039]) tensor(0.1566) tensor(0.0216)
tensor([0.2160, 0.0800]) tensor([0.0182, 0.0041]) tensor(0.1360) tensor(0.0193)
tensor([0.2189, 0.0736]) tensor([0.0183, 0.0034]) tensor(0.1453) tensor(0.0191)
tensor([0.1809, 0.0512]) tensor([0.0132, 0.0

In [0]:
print('mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))')
baseline = torch.Tensor(template_ch.baseline)
print([round(i,4) for i in torch.mean(baseline,dim=0).tolist()], [round(i,4) for i in torch.var(baseline,dim=0).tolist()], 
      round(float((baseline[:,0]-baseline[:,1]).mean()),4), round(float((baseline[:,0]-baseline[:,1]).var()),4), '\n')

for i in torch.Tensor(predictions_ch):
    print([round(i,4) for i in torch.mean(i,dim=0).tolist()], [round(i,4) for i in torch.var(i,dim=0).tolist()], 
          round(float((i[:,0]-i[:,1]).mean()),4), round(float((i[:,0]-i[:,1]).var()),4))

mean(prob(he/she)) var(prob(he/she)) mean(prob(he-she)) var(prob(he-she))
[0.237, 0.0718] [0.0206, 0.0037] 0.1652 0.0223 

[0.0961, 0.0297] [0.004, 0.0004] 0.0664 0.0032
[0.243, 0.0738] [0.0213, 0.0037] 0.1691 0.023
[0.2415, 0.073] [0.0211, 0.0037] 0.1685 0.0229
[0.2417, 0.073] [0.0213, 0.0037] 0.1687 0.023
[0.2443, 0.0749] [0.0217, 0.0038] 0.1694 0.0236
[0.2384, 0.0742] [0.0209, 0.0038] 0.1642 0.0226
[0.2273, 0.0777] [0.0195, 0.0041] 0.1496 0.0211
[0.2319, 0.0753] [0.02, 0.0039] 0.1566 0.0216
[0.216, 0.08] [0.0182, 0.0041] 0.136 0.0193
[0.2189, 0.0736] [0.0183, 0.0034] 0.1453 0.0191
[0.1809, 0.0512] [0.0132, 0.0017] 0.1296 0.0128
[0.1472, 0.0442] [0.0086, 0.0011] 0.1029 0.0077
[0.2165, 0.0643] [0.0177, 0.0029] 0.1522 0.0185


In [0]:
for i in range(len(template_ch.lines)):
    print("sentence: ", template_ch.lines[i])
    print("prediction: ", predictions_ch[0][i])
    print("baseline: ", template_ch.baseline[i], "\n")
